In [1]:
import os
import sys
import importlib
import pathlib as pl

In [2]:
import matplotlib
matplotlib.font_manager._rebuild()

AttributeError: module 'matplotlib' has no attribute 'font_manager'

In [3]:
import dask.dataframe as dd

os.environ["MODIN_ENGINE"] = "ray"
import modin.pandas as pd

import vaex

import multiprocessing

In [4]:
test = vaex.example()

In [5]:
test

#,id,x,y,z,vx,vy,vz,E,L,Lz,FeH
0,0,1.2318683862686157,-0.39692866802215576,-0.598057746887207,301.1552734375,174.05947875976562,27.42754554748535,-149431.40625,407.38897705078125,333.9555358886719,-1.0053852796554565
1,23,-0.16370061039924622,3.654221296310425,-0.25490644574165344,-195.00022888183594,170.47216796875,142.5302276611328,-124247.953125,890.2411499023438,684.6676025390625,-1.7086670398712158
2,32,-2.120255947113037,3.326052665710449,1.7078403234481812,-48.63423156738281,171.6472930908203,-2.079437255859375,-138500.546875,372.2410888671875,-202.17617797851562,-1.8336141109466553
3,8,4.7155890464782715,4.5852508544921875,2.2515437602996826,-232.42083740234375,-294.850830078125,62.85865020751953,-60037.0390625,1297.63037109375,-324.6875,-1.4786882400512695
4,16,7.21718692779541,11.99471664428711,-1.064562201499939,-1.6891745328903198,181.329345703125,-11.333610534667969,-83206.84375,1332.7989501953125,1328.948974609375,-1.8570483922958374
...,...,...,...,...,...,...,...,...,...,...,...
"329,995",21,1.9938701391220093,0.789276123046875,0.22205990552902222,-216.92990112304688,16.124420166015625,-211.244384765625,-146457.4375,457.72247314453125,203.36758422851562,-1.7451677322387695
"329,996",25,3.7180912494659424,0.721337616443634,1.6415337324142456,-185.92160034179688,-117.25082397460938,-105.4986572265625,-126627.109375,335.0025634765625,-301.8370056152344,-0.9822322130203247
"329,997",14,0.3688507676124573,13.029608726501465,-3.633934736251831,-53.677146911621094,-145.15771484375,76.70909881591797,-84912.2578125,817.1375732421875,645.8507080078125,-1.7645612955093384
"329,998",18,-0.11259264498949051,1.4529125690460205,2.168952703475952,179.30865478515625,205.79710388183594,-68.75872802734375,-133498.46875,724.000244140625,-283.6910400390625,-1.8808952569961548


In [ ]:
# ------- To load user-written script(s) -------
# # 1. To generate objects that will be used to load user-written script(s)
# ## Note: The numbers used below depend on the directory in which this script is
cwd = pl.Path.cwd()
path_project = cwd.parents[3]
path_hd = cwd.parents[2]

In [ ]:
# # 2. To load user-written script(s)
sys.path.append(str(path_hd))
os.chdir(path_project)
HD = importlib.import_module('H-Energy-Demand-Analysis')
FNC = importlib.import_module('F-Energy-Demand-Analysis_Common-Functions_Modin')
DD = importlib.import_module('D-Energy-Demand-Analysis_Data-Dictionary')

In [ ]:
# ------- To set path(s) -------
# # 1. Path to file(s) that will be loaded
# # 1.1. Path of SMUD ID Data
FILE_TO_LOAD_ID = 'SMUD_ID-Data.parquet'
PATH_TO_LOAD_ID = os.path.join(HD.PATH_DATA_INTERMEDIATE_SMUD_BILLING, FILE_TO_LOAD_ID)
# # 1.2. Path of SMUD Billing Data
FILE_TO_LOAD_BIllING = 'SMUD_Billing-Data.parquet'
PATH_TO_LOAD_BILLING = os.path.join(HD.PATH_DATA_INTERMEDIATE_SMUD_BILLING, FILE_TO_LOAD_BIllING)

# # # 2. Path to file(s) that will be saved as Parquet format
# (NOT Applicable)


# ------- To set path(s) parameter(s) -------
# # 0. Script Number
SCRIPT_NO = 'A-01-01A'

# # 1. To set parameters for Dask
N_PARTITIONS = multiprocessing.cpu_count()

In [ ]:
# ------- To load necessary dataset(s) -------
# # 1. SMUD ID Data
smud_id = dd.read_parquet(PATH_TO_LOAD_ID)

# # 2. SMUD Billing Data
smud_billing = dd.read_parquet(PATH_TO_LOAD_BILLING)

In [ ]:
# ------- To get basic information about ID dataset -------
# # 1. To look at the DF
smud_id.head(10)

In [ ]:
# # 2. To check columnes' data types
FNC.ddf_info(smud_id)

In [ ]:
# # 3. To find primary keys of the DF
# # 3.1. To check whether there are duplicated observations or not
smud_id.shape[0].compute() == smud_id.drop_duplicates().shape[0].compute()
# ## Note
# ## : The value of "True" means that there is no duplicated observations.

In [ ]:
# # 3.2. To find primary keys
list_cols_by = ['id_account', 'id_premise', 'rate_code_from_id', 'date_move_in', 'date_move_out'] 
test = smud_id.shape[0] == smud_id[list_cols_by].drop_duplicates().shape[0]
test.compute()
# ## Note
# ## : The value of "True" implies that there is no duplicated observations given the columns.

In [ ]:
# ------- To understand about the ID dataset -------
# # 1. To examine the relationship between `id_account` and `id_premise`
# # 1.2. Multiple `id_premise`s to a `id_account`?
list_cols_by = ['id_account', 'id_premise']
tmp_df = smud_id[list_cols_by].drop_duplicates()
test = tmp_df.shape[0] == tmp_df['id_account'].unique().size
test.compute()
# ## Note
# ## : The value of "False" means that there are `id_account`s having multiple `id_premise`s. This result seem to imply that the `id_account`s moved (to different `id_premise`s) several times.

In [ ]:
# # 1.2. Multiple `id_account`s to a `id_premise`?
list_cols_by = ['id_account', 'id_premise']
tmp_df = smud_id[list_cols_by].drop_duplicates()
test = tmp_df.shape[0] == tmp_df['id_premise'].unique().size
test.compute()
# ## Note
# ## : The value of "False" means that there are `id_premise`s having multiple `id_account`s. And this result seem to imply that the `id_premise`s have different `id_account`s as its residents. 

In [ ]:
# # 2. To examine how `id_premise` is related to `type_house` and `zip` (and `city`)
# # 2.1. Multiple `type_house`s to a `id_premise`?
list_cols_by = ['id_premise', 'type_house']
tmp_df = smud_id[list_cols_by].drop_duplicates()
test = tmp_df.shape[0] == tmp_df['id_premise'].unique().size
test.compute()
# ## Note
# ## : The value of "False" shows that several `id_premise`s have multiple `type_house`s.

In [ ]:
# # 2.2. Multiple `zip`s to a `id_premise`?
list_cols_by = ['id_premise', 'zip']
tmp_df = smud_id[list_cols_by].drop_duplicates()
test = tmp_df.shape[0] == tmp_df['id_premise'].unique().size
test.compute()
# ## Note
# ## : The value of "True" implies there is one-to-one relationship between `id_premise` and `zip`.

In [ ]:
# # 2.2.1. Multiple `zip`s to a `city`?
list_cols_by = ['city', 'zip']
tmp_df = smud_id[list_cols_by].drop_duplicates()
test = tmp_df.shape[0] == tmp_df['zip'].unique().size
test.compute()
# ## Note
# ## : The value of "False" implies there are several `zip`s having multiple `city`s.

In [ ]:
# ## For further investigation
grouped = tmp_df.groupby(['zip']).size()
grouped[grouped > 1].compute()

In [ ]:
list_zip = ['', '95652', '95683']
select = tmp_df['zip'].apply(lambda row: row in list_zip)
tmp_df[select].compute()
# ## Note:
# ## 1) "AFB" means "Air Force Base".
# ## 2) Rancho Murieta is near Sloughhouse.

In [ ]:
select = smud_id.eval('date_move_out < date_move_in')
smud_id[select].shape[0].compute()
# ## Note
# ## : This result shows that `date_move_out` >= `date_move_in` for all observations.

In [ ]:
# # 2. To examine how `is_in_isa` is related to `id_account` and `id_premise`
# # 2.1. Multiple `is_in_isa`s in a `id_account`?
list_cols_by = ['id_account', 'is_in_isa']
tmp_df = smud_id[list_cols_by].drop_duplicates()
test = tmp_df.shape[0] == tmp_df['id_account'].unique().size
test.compute()
# ## Note
# ## : The value of "True" means that there is one-to-one relationship between `id_account` and `is_in_isa`, which indicating whether a customer is in "ISA" (Interim Service Agreement).

In [ ]:
# # 2.2. Multiple `is_in_isa`s in a `id_premise`?
list_cols_by = ['id_premise', 'is_in_isa']
tmp_df = smud_id[list_cols_by].drop_duplicates()
test = tmp_df.shape[0] == tmp_df['id_premise'].unique().size
test.compute()
# ## Note
# ## : This value of "False" means that there are several `id_premise`s having multiple `is_in_isa`. 

In [ ]:
# # 2.3. For further investigation
# # 2.3.1. `id_account` having multiple `id_premise`s
list_cols_by = ['id_account', 'id_premise']
tmp_df = smud_id[list_cols_by].drop_duplicates().groupby(['id_account']).count()
select = tmp_df[tmp_df['id_premise'] > 1].index
print(smud_id.loc[select, 'is_in_isa'].value_counts().compute())
print('\n')

# # 2.3.2. `id_account` having just a `id_premise`
list_cols_by = ['id_account', 'id_premise']
tmp_df = smud_id[list_cols_by].drop_duplicates().groupby(['id_account']).count()
select = tmp_df[tmp_df['id_premise'] == 1].index
print(smud_id.loc[select, 'is_in_isa'].value_counts().compute())

In [ ]:
# # 2.3.3. `id_premise` having multiple `id_account`s
list_cols_by = ['id_account', 'id_premise']
tmp_df = smud_id[list_cols_by].drop_duplicates().groupby(['id_premise']).count()
select = tmp_df[tmp_df['id_account'] > 1].index
print(smud_id.loc[select, 'is_in_isa'].value_counts().compute())
print('\n')

# # 2.3.4. `id_premise` having just a `id_account`s
list_cols_by = ['id_account', 'id_premise']
tmp_df = smud_id[list_cols_by].drop_duplicates().groupby(['id_premise']).count()
select = tmp_df[tmp_df['id_account'] == 1].index
print(smud_id.loc[select, 'is_in_isa'].value_counts().compute())

In [ ]:
# # 3. To examine the relationship between `type_house` and `is_in_isa`
list_cols_by = ['type_house', 'is_in_isa']
smud_id[list_cols_by].drop_duplicates().groupby(['type_house']).count().compute()
# ## Note
# ## : This result implies that both house types have different values of `is_in_isa`.